<a href="https://colab.research.google.com/github/Ankitkumar1015/casia2.0/blob/main/Casia_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2

In [2]:
data_dir = '/content/drive/MyDrive/Ml/Casia 2.0'

In [3]:
def load_data(data_dir):
    images = []
    labels = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (128, 128))  # Resize images to 128x128
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)


In [5]:
images, labels = load_data(data_dir)

In [6]:
label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
labels = np.array([label_dict[label] for label in labels])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(label_dict))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(label_dict))

In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_dict), activation='softmax')
])

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [13]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
271/271 [==============================] - 311s 1s/step - loss: 2.1819 - accuracy: 0.1519 - val_loss: 2.1372 - val_accuracy: 0.1516
Epoch 2/25
271/271 [==============================] - 289s 1s/step - loss: 2.1436 - accuracy: 0.1502 - val_loss: 2.1320 - val_accuracy: 0.1419
Epoch 3/25
271/271 [==============================] - 318s 1s/step - loss: 2.1293 - accuracy: 0.1466 - val_loss: 2.1322 - val_accuracy: 0.1368
Epoch 4/25
271/271 [==============================] - 297s 1s/step - loss: 2.1303 - accuracy: 0.1448 - val_loss: 2.1356 - val_accuracy: 0.1331
Epoch 5/25
271/271 [==============================] - 294s 1s/step - loss: 2.1259 - accuracy: 0.1464 - val_loss: 2.1336 - val_accuracy: 0.1507
Epoch 6/25
271/271 [==============================] - 292s 1s/step - loss: 2.1243 - accuracy: 0.1524 - val_loss: 2.1308 - val_accuracy: 0.1540
Epoch 7/25
271/271 [==============================] - 292s 1s/step - loss: 2.1206 - accuracy: 0.1492 - val_loss: 2.1320 - val_accuracy: 0.1341

In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

68/68 [==============================] - 18s 258ms/step - loss: 2.1165 - accuracy: 0.1350
Test accuracy: 0.13499769568443298
